<a href="https://colab.research.google.com/github/whopriyam/Benchmarking-Transformer-Networks-for-Sexual-Harassment-Detection-on-Twitter/blob/main/Models/Electra_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers
# Optional: for Experiment management
!pip install wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Sarcasm.csv')

In [ ]:
df = df.filter(['Text','Label'], axis=1)
df

,Text,Label
0,He’d have my phone wedged up his ass sideways.,1
1,Yep 100% agree and the same with severine in s...,0
2,Stop having sex with men that won’t eat your p...,1
3,"When you see a sad man, just give him breast t...",1
4,We can't be together if you don't eat pussy,1
...,...,...
3599,only for a 5'7 brown hair brown eye gamer girl...,1
3600,"Don't be suspicious! Some butt, some suggestiv...",0
3601,mhmmmmm i hear ya big tittie lil booty,1
3602,I hate dirty stank ass cowboy fans bro lmao,0


In [ ]:
train_df = df[:int(0.8*len(df))]
test_df = df[int(0.8*len(df)):]


In [ ]:
import pandas as pd
import pytest

from simpletransformers.classification import (
    ClassificationModel,
    # MultiLabelClassificationModel,
)


model_type = "roberta"
model_name = "roberta-base"

In [ ]:
 model = ClassificationModel(
        model_type,
        model_name,
        cuda_device=0,
        args={
            'num_train_epochs': 5,
            "no_save": True,
            "reprocess_input_data": True,
            "overwrite_output_dir": True,
            "scheduler": "constant_schedule",
            "max_seq_length": 20,
        },
    )

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.

In [ ]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2883 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/361 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/361 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/361 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/361 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/361 [00:00<?, ?it/s]

(1805, 0.5293863684171166)

In [ ]:
len(test_df)

721

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/721 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/91 [00:00<?, ?it/s]

In [ ]:
list_eval = test_df['Text'].to_list()
type(list_eval)

list

In [ ]:
predictions, raw_outputs = model.predict(list_eval)

  0%|          | 0/721 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

In [ ]:
from sklearn import metrics


In [ ]:
targets = test_df['Label'].to_list()
metrics.accuracy_score(targets, predictions)

0.7725381414701803

In [ ]:
print(metrics.classification_report(targets, predictions))

              precision    recall  f1-score   support

           0       0.79      0.71      0.75       339
           1       0.76      0.83      0.79       382

    accuracy                           0.77       721
   macro avg       0.77      0.77      0.77       721
weighted avg       0.77      0.77      0.77       721

